In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import datetime

sys.path.insert(0, '../../../drlib')
import drlib as dr

%matplotlib widget

In [ ]:
#all 11776 spectra averaged together

antData     = np.load('../run1_antData_avgAll11776Spectra_gitignore.npy')
termData    = np.load('../run1_termData_avgAll11776Spectra_gitignore.npy')
diffData    = dr.fft2Watts(antData) - dr.fft2Watts(termData)

plt.close('all')
plt.figure()
plt.title('Difference Spectrum')
plt.xlabel('Frequency bins')
plt.ylabel('Power (watts)')
plt.plot(diffData[1:])

In [ ]:
#import stuff from the h5 file
database = pd.read_hdf('/drBiggerBoy/drData/run1Data/preProcDataSet.hdf5', 'database_DF')
f = h5py.File('/drBiggerBoy/drData/run1Data/preProcDataSet.hdf5', 'r')


In [ ]:
#pandas dataframe 
database.head()

In [ ]:
#returns the diff spec as an hdf object
#can use numpy indixing

#data is (freq, spec)  note measData is the index for spec]
plt.figure()
plt.plot(f['diffSpec_W'][1:,0:3])

In [ ]:
#slicing df
#example, where west is 140
database[database['West'] == 140]

In [ ]:
#get spectra where west = 140
#meas data is like an index for spectrum

#this is wrong. Something like this. Look up indexing numpy with array of indicies
f[1:, database[database['West'] == 140]['measData'].to_numpy()]

#want this to return an array of shape (~8.4M, 4032)

In [ ]:
#this is not quite right. Look up indexing df with date time
database[database < '2000-01-01']['measData'].to_numpy()

# Tasks

and plot average difference spectra from 7/7 12:46 to 12:50 and 7/9 from 7:00 to 8:00. two spectra

In [ ]:
#random example code that may or may not help

sigTimeArr = np.zeros(11776)
vetoTimeArr = np.zeros(11776)



measTimes = database.index.to_series()
deltas =measTimes.diff()
#print(measTimes)
#print(deltas)
gaps = deltas[deltas < datetime.timedelta(minutes=1)]
print(gaps.std())
print(gaps.mean())

plt.close('all')
plt.figure()
deltas.plot()
plt.ylabel('Difference (nSeconds)')
plt.figure()
gaps.plot()

def roachIdx2Rigol(roachIdx):
    numRoachBins    = int(2**23 - 1)
    numRigolBins    = 10000
    rigolIdx        = numRigolBins / numRoachBins * roachIdx
    return int(rigolIdx)

roachIdx = 2866101

for i in range(11776):
    sigTimeArr[i] = f['diffSpec_W'][roachIdx, i]
    vetoTimeArr[i] = f['vetoSpec_W'][roachIdx2Rigol(roachIdx), i]

print(sigTimeArr.mean())

plt.close('all')
plt.figure()
plt.plot(database.index, sigTimeArr, label = 'ROACH data (Gain corrected)')
plt.plot(database.index, vetoTimeArr, label = 'Rigol data (70dB isolation corrected)')
plt.xlabel('Datetime')
plt.ylabel('Linear Power')
plt.legend()